# Quickstart

The following data types are allowed: number, integer, string, array, object (corresponding to OpenAPIV2).

For example, to define a power plug scheme:

```yaml
group: digi.dev
version: v1
kind: Plug
control:
  power: string
``` 

Converted to kubernetes's resource:

```yaml
apiVersion: digi.dev/v1
kind: Plug        
metadata:
  name: plug-test
spec:
  control:        
    power:
      intent: "off"
```

# Simple Lamp digivice

In [1]:
from tutorial import (
    create,
    model_file,
    handler_file,
)
%elapsed_time

0:00:00


## Define a schema

In [2]:
%%elapsed_time

schema = """
group: digi.dev
version: v1
kind: Lamp
control:
  power: string
  brightness: number
"""

create(schema)

0:00:03


## Specify a model

In [3]:
m = model_file("lamp")

In [4]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Lamp         
metadata:
  name: lamp-test
spec:
  control:        
    power:
      intent: "on"
    brightness:
      intent: 0.8

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/lamp/deploy/cr.yaml
0:00:06


## Implement a driver

In [5]:
f = handler_file("lamp")

In [6]:
%%elapsed_time
%%writefile $f

from digi import on

@on.control("power")
def h(sv):
    sv["status"] = sv["intent"]

@on.control("brightness")
def h(sv):
    sv["status"] = sv["intent"]

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/lamp/driver/handler.py
0:00:08


## Build

In [7]:
!dq build lamp -q  # quiet
!dq image

IMAGE ID
lamp
motionsensor
room


## Run

In [8]:
!dq stop lamp lamp-test -q
!dq run lamp lamp-test

lamp-test


## Read status

In [9]:
!kubectl get lamp.digi.dev lamp-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Lamp
metadata:
  name: lamp-test
  namespace: default
spec:
  control:
    brightness:
      intent: 1
      status: 1
    power:
      intent: "on"
      status: "on"


## Update intent

In [10]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Lamp         
metadata:
  name: lamp-test
spec:
  control:        
    power:
      intent: "off"
    brightness:
      intent: 0.7

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/lamp/deploy/cr.yaml
0:00:22


In [11]:
!kubectl apply -f $m 2> /dev/null  
!kubectl get lamp lamp-test -oyaml | kubectl neat

lamp.digi.dev/lamp-test configured
apiVersion: digi.dev/v1
kind: Lamp
metadata:
  name: lamp-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.7
      status: 0.7
    power:
      intent: "off"
      status: "off"


In [12]:
# Alternatively, one can patch using a string (the previous method is preferred!), e.g.,
!kubectl patch lamp lamp-test -p '{"spec":{"control":{"power":{"intent":"off"}}}}' --type=merge

lamp.digi.dev/lamp-test patched (no change)


# HL abstraction: Room digivice

## Implementation

In [13]:
%%elapsed_time

# define schema
schema = """
group: digi.dev
version: v1
kind: Room
control:
  brightness: number
mount:     # mount reference
  digi.dev/v1/lamps: object
"""

create(schema)

0:00:34


In [14]:
# specify model
m = model_file("room")

In [15]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Room        
metadata:
  name: room-test
spec:
  control:        
    brightness:
      intent: 0.8

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/room/deploy/cr.yaml
0:00:38


In [16]:
# implement driver
f = handler_file("room")

In [17]:
%%elapsed_time
%%writefile $f

from digi import on, logger
from digi.view import TypeView, DotView

@on.mount
@on.control
def h(proc_view):    
    with TypeView(proc_view) as tv, DotView(tv) as dv:  
        logger.info(dv)
        room_brightness = dv.root.control.brightness
        room_brightness.status = 0
        
        if "lamps" not in dv:
            return
        
        # handle status
        active_lamps = [l for _, l in dv.lamps.items() 
                        if l.control.power.status == "on"]
        for l in active_lamps:
            room_brightness.status += l.control.brightness.status
            
        # handle intent
        for l in dv.lamps.items:
            room_brightness.status += l.control.brightness.status
            l.control.brightness.intent = room_brightness.intent / len(active_lamps)

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/room/driver/handler.py
0:00:41


In [18]:
# build and run
!dq build room -q
!dq stop room room-test || true
!dq run room room-test

room-test
room-test


## Debug

In [19]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8


In [20]:
!dq log room-test

[2021-04-18 19:25:49,128] digi                 [INFO    ] Started an operator
[2021-04-18 19:25:49,129] digi.mount           [INFO    ] Started the mounter
[2021-04-18 19:25:49,129] digi.mount           [INFO    ] Started the mounter
[2021-04-18 19:25:49,132] digi                 [INFO    ] Started an operator
[2021-04-18 19:25:49,136] kopf.reactor.running [WARNING ] OS signals are ignored: running not in the main thread.
[2021-04-18 19:25:49,565] digi                 [INFO    ] {'root': {'control': {'brightness': {'intent': 0.8}}}}
[2021-04-18 19:25:49,565] digi                 [INFO    ] {'root': {'control': {'brightness': {'intent': 0.8}}}}
[2021-04-18 19:25:49,566] digi                 [INFO    ] {'root': {'control': {'brightness': {'intent': 0.8, 'status': 0}}}}
[2021-04-18 19:25:49,566] digi                 [INFO    ] {'root': {'control': {'brightness': {'intent': 0.8, 'status': 0}}}}
[2021-04-18 19:25:49,598] digi.main            [INFO    ] Done reconciliation
[2021-04-18 19:25:

## Mount lamps

In [21]:
!dq mount lamp-test room-test

In [22]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8
      status: 0
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              status: 0.7
            power:
              status: "off"
        status: active


Or [watch the changes](http://localhost:8881/notebooks/display.ipynb).

## Playing with room brightness

In [23]:
!kubectl patch room room-test -p '{"spec":{"control":{"brightness":{"intent":1}}}}' --type=merge

room.digi.dev/room-test patched


In [24]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 1
      status: 0
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              status: 0.7
            power:
              status: "off"
        status: active


## Connect to physical lamps

...

# Activate the Room with motion

## Pull the MotionSensor

In [25]:
!dq pull motionsensor

motionsensor


In [26]:
# run the motion sensor
!dq stop motionsensor motion-test || true
!dq run motionsensor motion-test

motion-test
motion-test


In [27]:
!kubectl get motionsensor motion-test -oyaml | kubectl neat

apiVersion: mock.digi.dev/v1
kind: MotionSensor
metadata:
  name: motion-test
  namespace: default
spec:
  control:
    sensitivity:
      intent: 1
      status: 1
  obs:
    battery_level: 100%


In [28]:
# update its sensitivity
!kubectl patch motionsensor motion-test -p '{"spec":{"control":{"sensitivity":{"intent":10}}}}' --type=merge

motionsensor.mock.digi.dev/motion-test patched


In [29]:
!kubectl get motionsensor motion-test -oyaml | kubectl neat

apiVersion: mock.digi.dev/v1
kind: MotionSensor
metadata:
  name: motion-test
  namespace: default
spec:
  control:
    sensitivity:
      intent: 10
      status: 10
  obs:
    battery_level: 100%
    last_triggered_time: 1.6187739951085074e+09


## Modify Room

In [30]:
# Update the Room's schema s.t. it can mount the motionsensor
# TBD

## Mount a motionsensor

In [ ]:
!dq mount mostionsensor-test room-test

## Add Reflex

A reflex is an on-model policy that can be declared on the model to XYZ

In [ ]:
# specify model
m = model_file("room")

In [ ]:
%%elapsed_time
%%writefile $f

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
spec:
  control:
    brightness:
      intent: 0.8
    mode:
      intent: idle
  reflex:
    motion-mode:
      policy: 'if $time - ."motionsensor-test".obs.last_triggered_time <= 600
                           then .root.control.mode.intent = "work" else . end'
      priority: -1
      processor: jq

## Connect to real motion detectors

...

# Home (Bonus)

## Pull the base image

In [ ]:
!dq pull home

## Modify the Home driver

Goal: Home have a "mode" control attribute that allows one to tune it to predefined modes. Each mode decides the brightness of the Rooms that mounted to the Home.

In [ ]:
f = handler_file("home")

In [ ]:
%%elapsed_time
%%writefile $f

# TBD
import digi
import digi.on as on

# validation
@on.attr
def h():
    ...

# intent back-prop
@on.mount
def h():
    ...

# status
@on.mount
def h():
    ...

# intent
@on.mount
@on.control
def h():
    ...

In [ ]:
# build and run
!dq build home -q
!dq stop home home-test || true
!dq run home home-test

## Mount rooms

In [ ]:
!dq mount room-test home-test

In [ ]:
# set the modes
!kubectl patch home home-test -p '{"spec":{"control":{"mode":{"intent":YOURMODE}}}}' --type=merge